## 🔧 Configuración del Entorno

Importamos librerías y configuramos el entorno de trabajo.

In [1]:
# Importar librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from scipy import stats

warnings.filterwarnings('ignore')

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Librerías importadas exitosamente")

✅ Librerías importadas exitosamente


In [3]:
# Cargar contexto de Kedro
try:
    import os
    from kedro.framework.session import KedroSession
    from kedro.framework.startup import bootstrap_project

    # Cambiar al directorio raíz del proyecto
    original_dir = os.getcwd()
    project_path = os.path.dirname(os.getcwd())
    os.chdir(project_path)
    
    bootstrap_project(project_path)
    session = KedroSession.create()
    context = session.load_context()
    catalog = context.catalog
    
    print("✅ Contexto Kedro cargado exitosamente")
    os.chdir(original_dir)
    
except Exception as e:
    print(f"⚠️ Error cargando Kedro: {e}")
    print("Continuando sin contexto Kedro...")
    catalog = None

[09/21/25 22:16:42] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=964027;file://C:\Users\cmuru\OneDrive\Escritorio\datos_covid\kedro-env\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=882085;file://C:\Users\cmuru\OneDrive\Escritorio\datos_covid\kedro-env\Lib\site-packages\kedro_telemetry\plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

✅ Contexto Kedro cargado exitosamente


## 📊 3.1 Carga de Datos

Cargamos los datos procesados de las fases anteriores.

In [4]:
# Cargar datos desde catálogo Kedro o crear datos de ejemplo
if catalog is not None:
    try:
        # Intentar cargar datos procesados
        df_trabajo = catalog.load("primary_covid_complete")
        print(f"✅ Datos cargados desde Kedro: {df_trabajo.shape}")
    except Exception as e:
        print(f"⚠️ Error cargando datos procesados: {e}")
        try:
            # Fallback: cargar datos raw
            df_2020 = catalog.load("raw_covid_2020")
            df_2021 = catalog.load("raw_covid_2021") 
            df_2022 = catalog.load("raw_covid_2022")
            df_trabajo = pd.concat([df_2020, df_2021, df_2022], ignore_index=True)
            print(f"✅ Datos raw concatenados: {df_trabajo.shape}")
        except:
            df_trabajo = None
else:
    df_trabajo = None

# Crear datos de ejemplo si no hay datos disponibles
if df_trabajo is None or df_trabajo.empty:
    print("⚠️ Creando datos de ejemplo para demostración...")
    
    # Generar datos sintéticos para COVID-19 Chile
    dates = pd.date_range('2020-01-01', '2022-12-31', freq='D')
    n_days = len(dates)
    
    # Simular datos realistas de COVID-19
    np.random.seed(42)
    
    # Casos nuevos con tendencias estacionales
    base_cases = 50 + 200 * np.sin(np.arange(n_days) * 2 * np.pi / 365.25) + np.random.exponential(30, n_days)
    # Agregar olas pandémicas
    wave1 = np.where((dates >= '2020-06-01') & (dates <= '2020-09-01'), 
                     np.random.exponential(100, n_days), 0)
    wave2 = np.where((dates >= '2021-03-01') & (dates <= '2021-06-01'), 
                     np.random.exponential(150, n_days), 0)
    
    new_confirmed = np.maximum(0, base_cases + wave1 + wave2).astype(int)
    cumulative_confirmed = np.cumsum(new_confirmed)
    
    # Muertes (aproximadamente 2% de casos con retraso)
    new_deceased = np.maximum(0, np.random.poisson(new_confirmed * 0.02, n_days))
    cumulative_deceased = np.cumsum(new_deceased)
    
    # Crear DataFrame
    df_trabajo = pd.DataFrame({
        'date': dates,
        'location_key': 'CL',
        'new_confirmed': new_confirmed,
        'cumulative_confirmed': cumulative_confirmed,
        'new_deceased': new_deceased,
        'cumulative_deceased': cumulative_deceased,
        'population': 19116201,  # Población Chile aproximada
        'region': 'Nacional'
    })
    
    print(f"✅ Datos sintéticos creados: {df_trabajo.shape}")

# Verificar datos
print(f"\n📊 Dataset de trabajo:")
print(f"   • Forma: {df_trabajo.shape}")
print(f"   • Columnas: {list(df_trabajo.columns)}")
print(f"   • Rango temporal: {df_trabajo['date'].min()} a {df_trabajo['date'].max()}")

[09/21/25 22:16:48] INFO     Loading data from primary_covid_complete (CSVDataset)...          ]8;id=911989;file://C:\Users\cmuru\OneDrive\Escritorio\datos_covid\kedro-env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=855980;file://C:\Users\cmuru\OneDrive\Escritorio\datos_covid\kedro-env\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

✅ Datos cargados desde Kedro: (99193, 15)

📊 Dataset de trabajo:
   • Forma: (99193, 15)
   • Columnas: ['location_key', 'new_recovered', 'year', 'new_deceased', 'cumulative_recovered', 'new_tested', 'date', 'cumulative_confirmed', 'new_confirmed', 'cumulative_tested', 'cumulative_deceased', 'month', 'quarter', 'day_of_week', 'week_of_year']
   • Rango temporal: 2020-01-01 00:00:00 a 2022-09-13 00:00:00


## 🧹 3.2 Limpieza de Datos

### 3.2.1 Análisis y Tratamiento de Valores Faltantes

In [5]:
def analizar_valores_faltantes(df):
    """
    Analiza patrones de valores faltantes en detalle.
    """
    print("\n🔍 ANÁLISIS DE VALORES FALTANTES")
    print("=" * 50)
    
    if df.empty:
        print("❌ Dataset vacío")
        return {}
    
    # Análisis de completitud
    missing_info = {}
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / len(df)) * 100
    
    print(f"📊 Resumen general:")
    print(f"   • Total registros: {len(df):,}")
    print(f"   • Columnas con datos faltantes: {(missing_count > 0).sum()}")
    
    for col in df.columns:
        if missing_count[col] > 0:
            pct = missing_percent[col]
            
            # Clasificar severidad
            if pct >= 50:
                nivel = "🔴 CRÍTICO"
                estrategia = "Considerar eliminar columna"
            elif pct >= 20:
                nivel = "🟡 ALTO"
                estrategia = "Imputación avanzada necesaria"
            elif pct >= 5:
                nivel = "🟠 MODERADO"
                estrategia = "Imputación estándar"
            else:
                nivel = "🟢 BAJO"
                estrategia = "Imputación simple"
            
            missing_info[col] = {
                'count': int(missing_count[col]),
                'percent': pct,
                'nivel': nivel,
                'estrategia_recomendada': estrategia
            }
            
            print(f"   • {col}: {missing_count[col]} ({pct:.1f}%) - {nivel}")
    
    if not missing_info:
        print("✅ No se encontraron valores faltantes")
    
    return missing_info

# Analizar valores faltantes
missing_analysis = analizar_valores_faltantes(df_trabajo)


🔍 ANÁLISIS DE VALORES FALTANTES
📊 Resumen general:
   • Total registros: 99,193
   • Columnas con datos faltantes: 0
✅ No se encontraron valores faltantes


In [6]:
def aplicar_estrategias_imputacion(df, missing_info):
    """
    Aplica estrategias diferenciadas de imputación según el tipo de variable.
    """
    print("\n🔧 APLICACIÓN DE ESTRATEGIAS DE IMPUTACIÓN")
    print("=" * 50)
    
    df_imputado = df.copy()
    estrategias_aplicadas = {}
    
    if not missing_info:
        print("✅ No hay valores faltantes que tratar")
        return df_imputado, estrategias_aplicadas
    
    for col, info in missing_info.items():
        if col not in df_imputado.columns:
            continue
            
        pct_missing = info['percent']
        print(f"\n📊 Tratando columna: {col} ({pct_missing:.1f}% faltante)")
        
        # Determinar estrategia según tipo de variable y porcentaje faltante
        if pct_missing >= 50:
            print(f"   ⚠️ Demasiados valores faltantes. Marcando para eliminación.")
            estrategias_aplicadas[col] = "ELIMINAR"
            continue
        
        # Identificar tipo de variable
        if df_imputado[col].dtype in ['object', 'category']:
            # Variables categóricas
            estrategia = "moda"
            valor_imputacion = df_imputado[col].mode()[0] if len(df_imputado[col].mode()) > 0 else "MISSING"
            df_imputado[col] = df_imputado[col].fillna(valor_imputacion)
            estrategias_aplicadas[col] = f"Imputación por moda: {valor_imputacion}"
            
        elif 'date' in col.lower():
            # Variables de fecha
            estrategia = "interpolación_temporal"
            df_imputado[col] = pd.to_datetime(df_imputado[col])
            df_imputado[col] = df_imputado[col].interpolate(method='time')
            estrategias_aplicadas[col] = "Interpolación temporal"
            
        else:
            # Variables numéricas
            if pct_missing < 5:
                # Imputación simple por mediana
                estrategia = "mediana"
                valor_mediana = df_imputado[col].median()
                df_imputado[col] = df_imputado[col].fillna(valor_mediana)
                estrategias_aplicadas[col] = f"Imputación por mediana: {valor_mediana:.2f}"
                
            elif pct_missing < 20:
                # Imputación por media si es distribución normal
                if abs(df_imputado[col].skew()) < 1:  # Distribución aproximadamente normal
                    estrategia = "media"
                    valor_media = df_imputado[col].mean()
                    df_imputado[col] = df_imputado[col].fillna(valor_media)
                    estrategias_aplicadas[col] = f"Imputación por media: {valor_media:.2f}"
                else:
                    # Usar mediana para distribuciones asimétricas
                    estrategia = "mediana"
                    valor_mediana = df_imputado[col].median()
                    df_imputado[col] = df_imputado[col].fillna(valor_mediana)
                    estrategias_aplicadas[col] = f"Imputación por mediana: {valor_mediana:.2f}"
            else:
                # Imputación avanzada con KNN
                try:
                    estrategia = "KNN"
                    # Preparar datos para KNN (solo variables numéricas)
                    numeric_cols = df_imputado.select_dtypes(include=[np.number]).columns
                    df_numeric = df_imputado[numeric_cols].copy()
                    
                    # Aplicar KNN imputer
                    imputer = KNNImputer(n_neighbors=5)
                    df_numeric_imputed = pd.DataFrame(
                        imputer.fit_transform(df_numeric),
                        columns=df_numeric.columns,
                        index=df_numeric.index
                    )
                    
                    df_imputado[col] = df_numeric_imputed[col]
                    estrategias_aplicadas[col] = "Imputación KNN (k=5)"
                except Exception as e:
                    # Fallback a mediana
                    print(f"   ⚠️ Error en KNN, usando mediana: {e}")
                    valor_mediana = df_imputado[col].median()
                    df_imputado[col] = df_imputado[col].fillna(valor_mediana)
                    estrategias_aplicadas[col] = f"Fallback mediana: {valor_mediana:.2f}"
        
        print(f"   ✅ Aplicada: {estrategias_aplicadas[col]}")
    
    # Eliminar columnas marcadas para eliminación
    cols_eliminar = [col for col, estrategia in estrategias_aplicadas.items() if estrategia == "ELIMINAR"]
    if cols_eliminar:
        df_imputado = df_imputado.drop(columns=cols_eliminar)
        print(f"\n🗑️ Columnas eliminadas: {cols_eliminar}")
    
    print(f"\n📊 Resultado de imputación:")
    print(f"   • Dataset original: {df.shape}")
    print(f"   • Dataset imputado: {df_imputado.shape}")
    print(f"   • Estrategias aplicadas: {len(estrategias_aplicadas)}")
    
    return df_imputado, estrategias_aplicadas

# Aplicar estrategias de imputación
df_trabajo_limpio, estrategias = aplicar_estrategias_imputacion(df_trabajo, missing_analysis)


🔧 APLICACIÓN DE ESTRATEGIAS DE IMPUTACIÓN
✅ No hay valores faltantes que tratar


### 3.2.2 Detección y Tratamiento de Outliers

In [7]:
def detectar_y_tratar_outliers(df):
    """
    Detecta outliers usando múltiples métodos y aplica tratamiento diferenciado.
    """
    print("\n🎯 DETECCIÓN Y TRATAMIENTO DE OUTLIERS")
    print("=" * 50)
    
    df_outliers = df.copy()
    outliers_treatment = {}
    
    # Variables numéricas para análisis
    vars_numericas = df_outliers.select_dtypes(include=[np.number]).columns.tolist()
    # Excluir variables que no deben tratarse como outliers
    vars_excluir = ['year', 'month', 'day', 'population']
    vars_analizar = [var for var in vars_numericas if var not in vars_excluir]
    
    print(f"📊 Variables a analizar: {len(vars_analizar)}")
    
    for var in vars_analizar[:5]:  # Limitar para demostración
        if var not in df_outliers.columns:
            continue
            
        serie_original = df_outliers[var].dropna()
        if len(serie_original) < 10:
            continue
            
        print(f"\n📈 Analizando: {var}")
        
        # Método 1: IQR
        Q1 = serie_original.quantile(0.25)
        Q3 = serie_original.quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        outliers_iqr = (serie_original < limite_inferior) | (serie_original > limite_superior)
        
        # Método 2: Z-Score
        z_scores = np.abs(stats.zscore(serie_original))
        outliers_zscore = z_scores > 3
        
        # Combinar métodos
        outliers_combinados = outliers_iqr | outliers_zscore
        pct_outliers = outliers_combinados.sum() / len(serie_original) * 100
        
        print(f"   • Outliers detectados: {outliers_combinados.sum()} ({pct_outliers:.1f}%)")
        
        # Estrategia de tratamiento según porcentaje
        if pct_outliers <= 1:
            # Pocos outliers: eliminar
            mask_clean = ~outliers_combinados
            valores_limpios = serie_original[mask_clean]
            df_outliers.loc[df_outliers[var].notna(), var] = valores_limpios.reindex(
                df_outliers.loc[df_outliers[var].notna()].index, fill_value=np.nan)
            outliers_treatment[var] = f"Eliminación directa ({outliers_combinados.sum()} valores)"
            
        elif pct_outliers <= 5:
            # Moderados outliers: winsorización
            limite_inf_wins = serie_original.quantile(0.05)
            limite_sup_wins = serie_original.quantile(0.95)
            df_outliers[var] = df_outliers[var].clip(lower=limite_inf_wins, upper=limite_sup_wins)
            outliers_treatment[var] = f"Winsorización (5%-95%)"
            
        elif pct_outliers <= 15:
            # Muchos outliers: transformación log
            if (serie_original > 0).all():
                df_outliers[var] = np.log1p(df_outliers[var])
                outliers_treatment[var] = f"Transformación logarítmica"
            else:
                # Si hay valores <= 0, usar winsorización
                limite_inf_wins = serie_original.quantile(0.01)
                limite_sup_wins = serie_original.quantile(0.99)
                df_outliers[var] = df_outliers[var].clip(lower=limite_inf_wins, upper=limite_sup_wins)
                outliers_treatment[var] = f"Winsorización (1%-99%)"
        else:
            # Demasiados outliers: solo winsorización suave
            limite_inf_wins = serie_original.quantile(0.01)
            limite_sup_wins = serie_original.quantile(0.99)
            df_outliers[var] = df_outliers[var].clip(lower=limite_inf_wins, upper=limite_sup_wins)
            outliers_treatment[var] = f"Winsorización suave (1%-99%)"
        
        print(f"   ✅ Tratamiento: {outliers_treatment[var]}")
    
    print(f"\n📊 Resumen tratamiento outliers:")
    print(f"   • Variables analizadas: {len(vars_analizar[:5])}")
    print(f"   • Variables tratadas: {len(outliers_treatment)}")
    
    return df_outliers, outliers_treatment

# Detectar y tratar outliers
df_trabajo_limpio, outliers_treatment = detectar_y_tratar_outliers(df_trabajo_limpio)


🎯 DETECCIÓN Y TRATAMIENTO DE OUTLIERS
📊 Variables a analizar: 11

📈 Analizando: new_recovered
   • Outliers detectados: 77 (0.1%)
   ✅ Tratamiento: Eliminación directa (77 valores)

📈 Analizando: new_deceased
   • Outliers detectados: 8986 (9.1%)
   ✅ Tratamiento: Winsorización (1%-99%)

📈 Analizando: cumulative_recovered
   • Outliers detectados: 77 (0.1%)
   ✅ Tratamiento: Eliminación directa (77 valores)

📈 Analizando: new_tested
   • Outliers detectados: 8738 (8.8%)
   ✅ Tratamiento: Winsorización (1%-99%)

📈 Analizando: cumulative_confirmed
   • Outliers detectados: 14533 (14.7%)
   ✅ Tratamiento: Winsorización (1%-99%)

📊 Resumen tratamiento outliers:
   • Variables analizadas: 5
   • Variables tratadas: 5


## 🔧 3.3 Feature Engineering Avanzado

### 3.3.1 Variables Temporales

In [8]:
def crear_features_temporales(df):
    """
    Crea features avanzadas basadas en información temporal.
    """
    print("\n⏰ CREACIÓN DE FEATURES TEMPORALES")
    print("=" * 50)
    
    df_temporal = df.copy()
    
    if 'date' in df_temporal.columns:
        # Asegurar que date es datetime
        df_temporal['date'] = pd.to_datetime(df_temporal['date'])
        
        # Features básicas de fecha
        df_temporal['year'] = df_temporal['date'].dt.year
        df_temporal['month'] = df_temporal['date'].dt.month
        df_temporal['day'] = df_temporal['date'].dt.day
        df_temporal['day_of_week'] = df_temporal['date'].dt.dayofweek  # 0=Monday
        df_temporal['day_of_year'] = df_temporal['date'].dt.dayofyear
        df_temporal['week_of_year'] = df_temporal['date'].dt.isocalendar().week
        df_temporal['quarter'] = df_temporal['date'].dt.quarter
        
        # Features cíclicas (para capturar naturaleza cíclica del tiempo)
        df_temporal['month_sin'] = np.sin(2 * np.pi * df_temporal['month'] / 12)
        df_temporal['month_cos'] = np.cos(2 * np.pi * df_temporal['month'] / 12)
        df_temporal['day_of_week_sin'] = np.sin(2 * np.pi * df_temporal['day_of_week'] / 7)
        df_temporal['day_of_week_cos'] = np.cos(2 * np.pi * df_temporal['day_of_week'] / 7)
        
        # Features especiales
        df_temporal['is_weekend'] = (df_temporal['day_of_week'] >= 5).astype(int)
        df_temporal['is_month_start'] = (df_temporal['day'] <= 7).astype(int)
        df_temporal['is_month_end'] = (df_temporal['day'] >= 25).astype(int)
        df_temporal['is_quarter_start'] = ((df_temporal['month'] % 3 == 1) & (df_temporal['day'] <= 7)).astype(int)
        
        # Días desde el inicio de la pandemia
        fecha_inicio_pandemia = pd.to_datetime('2020-03-01')  # Aproximado para Chile
        df_temporal['days_since_pandemic_start'] = (df_temporal['date'] - fecha_inicio_pandemia).dt.days
        
        # Período de la pandemia
        def asignar_periodo_pandemia(fecha):
            if fecha < pd.to_datetime('2020-06-01'):
                return 'inicial'
            elif fecha < pd.to_datetime('2021-01-01'):
                return 'primera_ola'
            elif fecha < pd.to_datetime('2021-07-01'):
                return 'vacunacion_inicial'
            elif fecha < pd.to_datetime('2022-01-01'):
                return 'segunda_ola'
            else:
                return 'endemica'
        
        df_temporal['pandemic_period'] = df_temporal['date'].apply(asignar_periodo_pandemia)
        
        print(f"✅ Features temporales creadas: 16 nuevas variables")
        print(f"   • Básicas: year, month, day, day_of_week, day_of_year, week_of_year, quarter")
        print(f"   • Cíclicas: month_sin/cos, day_of_week_sin/cos")
        print(f"   • Especiales: is_weekend, is_month_start/end, is_quarter_start")
        print(f"   • Pandemia: days_since_pandemic_start, pandemic_period")
        
        return df_temporal
    else:
        print("❌ No se encontró columna 'date' para features temporales")
        return df_temporal

# Crear features temporales
df_con_features_temporales = crear_features_temporales(df_trabajo_limpio)


⏰ CREACIÓN DE FEATURES TEMPORALES
✅ Features temporales creadas: 16 nuevas variables
   • Básicas: year, month, day, day_of_week, day_of_year, week_of_year, quarter
   • Cíclicas: month_sin/cos, day_of_week_sin/cos
   • Especiales: is_weekend, is_month_start/end, is_quarter_start
   • Pandemia: days_since_pandemic_start, pandemic_period


### 3.3.2 Variables de Tendencias y Lags

In [9]:
def crear_features_tendencias_lags(df):
    """
    Crea features de tendencias, lags y rolling statistics.
    """
    print("\n📈 CREACIÓN DE FEATURES DE TENDENCIAS Y LAGS")
    print("=" * 50)
    
    df_trends = df.copy()
    
    # Variables numéricas para análisis de tendencias
    vars_numericas = df_trends.select_dtypes(include=[np.number]).columns
    vars_covid = [col for col in vars_numericas if any(keyword in col.lower() 
                  for keyword in ['confirmed', 'deceased', 'recovered', 'tested'])]
    
    if len(vars_covid) > 0 and 'date' in df_trends.columns:
        # Ordenar por fecha
        df_trends = df_trends.sort_values('date').reset_index(drop=True)
        
        features_creadas = []
        
        for var in vars_covid:
            if var in df_trends.columns:
                print(f"\n📊 Procesando variable: {var}")
                
                # 1. Variables de Lag (valores pasados)
                for lag in [1, 3, 7, 14]:
                    lag_col = f"{var}_lag_{lag}"
                    df_trends[lag_col] = df_trends[var].shift(lag)
                    features_creadas.append(lag_col)
                
                # 2. Rolling Statistics (ventanas móviles)
                for window in [3, 7, 14, 30]:
                    # Media móvil
                    rolling_mean_col = f"{var}_rolling_mean_{window}"
                    df_trends[rolling_mean_col] = df_trends[var].rolling(window=window, min_periods=1).mean()
                    features_creadas.append(rolling_mean_col)
                    
                    # Desviación estándar móvil
                    rolling_std_col = f"{var}_rolling_std_{window}"
                    df_trends[rolling_std_col] = df_trends[var].rolling(window=window, min_periods=1).std()
                    features_creadas.append(rolling_std_col)
                    
                    # Máximo y mínimo móvil
                    rolling_max_col = f"{var}_rolling_max_{window}"
                    rolling_min_col = f"{var}_rolling_min_{window}"
                    df_trends[rolling_max_col] = df_trends[var].rolling(window=window, min_periods=1).max()
                    df_trends[rolling_min_col] = df_trends[var].rolling(window=window, min_periods=1).min()
                    features_creadas.extend([rolling_max_col, rolling_min_col])
                
                # 3. Variables de Cambio (diferencias y tasas)
                diff_1_col = f"{var}_diff_1"
                diff_7_col = f"{var}_diff_7"
                df_trends[diff_1_col] = df_trends[var].diff(1)
                df_trends[diff_7_col] = df_trends[var].diff(7)
                features_creadas.extend([diff_1_col, diff_7_col])
                
                # Tasa de cambio porcentual
                pct_change_1_col = f"{var}_pct_change_1"
                pct_change_7_col = f"{var}_pct_change_7"
                df_trends[pct_change_1_col] = df_trends[var].pct_change(1)
                df_trends[pct_change_7_col] = df_trends[var].pct_change(7)
                features_creadas.extend([pct_change_1_col, pct_change_7_col])
                
                # 4. Variables de Aceleración (segunda derivada)
                acceleration_col = f"{var}_acceleration"
                df_trends[acceleration_col] = df_trends[diff_1_col].diff(1)
                features_creadas.append(acceleration_col)
                
                # 5. Variables de Posición Relativa
                relative_pos_col = f"{var}_relative_position_30d"
                rolling_min_30 = df_trends[var].rolling(window=30, min_periods=1).min()
                rolling_max_30 = df_trends[var].rolling(window=30, min_periods=1).max()
                df_trends[relative_pos_col] = ((df_trends[var] - rolling_min_30) / 
                                              (rolling_max_30 - rolling_min_30 + 1e-8))
                features_creadas.append(relative_pos_col)
                
                print(f"   ✅ {len([f for f in features_creadas if var in f])} features creadas")
        
        print(f"\n📊 Total de features de tendencias creadas: {len(features_creadas)}")
        
        return df_trends, features_creadas
    else:
        print("❌ No se encontraron variables COVID o columna 'date'")
        return df_trends, []

# Crear features de tendencias y lags
df_con_trends, features_trends = crear_features_tendencias_lags(df_con_features_temporales)


📈 CREACIÓN DE FEATURES DE TENDENCIAS Y LAGS

📊 Procesando variable: new_recovered
   ✅ 26 features creadas

📊 Procesando variable: new_deceased
   ✅ 26 features creadas

📊 Procesando variable: cumulative_recovered
   ✅ 26 features creadas

📊 Procesando variable: new_tested
   ✅ 26 features creadas

📊 Procesando variable: cumulative_confirmed
   ✅ 26 features creadas

📊 Procesando variable: new_confirmed
   ✅ 26 features creadas

📊 Procesando variable: cumulative_tested
   ✅ 26 features creadas

📊 Procesando variable: cumulative_deceased
   ✅ 26 features creadas

📊 Total de features de tendencias creadas: 208


### 3.3.3 Variables de Ratios y Proporciones

In [10]:
def crear_features_ratios(df):
    """
    Crea features basadas en ratios y proporciones epidemiológicas.
    """
    print("\n🔢 CREACIÓN DE FEATURES DE RATIOS Y PROPORCIONES")
    print("=" * 50)
    
    df_ratios = df.copy()
    features_ratios = []
    
    # Encontrar variables principales
    new_confirmed = None
    cumulative_confirmed = None
    new_deceased = None
    cumulative_deceased = None
    
    for col in df_ratios.columns:
        if 'new_confirmed' in col and not any(x in col for x in ['lag', 'rolling', 'diff']):
            new_confirmed = col
        elif 'cumulative_confirmed' in col and not any(x in col for x in ['lag', 'rolling', 'diff']):
            cumulative_confirmed = col
        elif 'new_deceased' in col and not any(x in col for x in ['lag', 'rolling', 'diff']):
            new_deceased = col
        elif 'cumulative_deceased' in col and not any(x in col for x in ['lag', 'rolling', 'diff']):
            cumulative_deceased = col
    
    print(f"Variables identificadas:")
    print(f"   • Casos nuevos: {new_confirmed}")
    print(f"   • Casos acumulados: {cumulative_confirmed}")
    print(f"   • Muertes nuevas: {new_deceased}")
    print(f"   • Muertes acumuladas: {cumulative_deceased}")
    
    # 1. RATIOS EPIDEMIOLÓGICOS BÁSICOS
    if new_deceased and new_confirmed:
        # Tasa de letalidad diaria (CFR diario)
        cfr_daily_col = 'case_fatality_rate_daily'
        df_ratios[cfr_daily_col] = (df_ratios[new_deceased] / 
                                   (df_ratios[new_confirmed] + 1e-8)) * 100
        features_ratios.append(cfr_daily_col)
    
    if cumulative_deceased and cumulative_confirmed:
        # Tasa de letalidad acumulada (CFR total)
        cfr_total_col = 'case_fatality_rate_total'
        df_ratios[cfr_total_col] = (df_ratios[cumulative_deceased] / 
                                   (df_ratios[cumulative_confirmed] + 1e-8)) * 100
        features_ratios.append(cfr_total_col)
    
    # 2. RATIOS DE CRECIMIENTO
    if new_confirmed:
        # Ratio de casos nuevos vs promedio de la semana anterior
        if f"{new_confirmed}_rolling_mean_7" in df_ratios.columns:
            growth_ratio_col = 'weekly_growth_ratio'
            rolling_mean_7d_lag = df_ratios[f"{new_confirmed}_rolling_mean_7"].shift(7)
            df_ratios[growth_ratio_col] = (df_ratios[f"{new_confirmed}_rolling_mean_7"] / 
                                          (rolling_mean_7d_lag + 1e-8))
            features_ratios.append(growth_ratio_col)
    
    # 3. RATIOS DE VOLATILIDAD
    if new_confirmed:
        if f"{new_confirmed}_rolling_mean_7" in df_ratios.columns and f"{new_confirmed}_rolling_std_7" in df_ratios.columns:
            cv_col = 'cases_coefficient_variation_7d'
            df_ratios[cv_col] = (df_ratios[f"{new_confirmed}_rolling_std_7"] / 
                                (df_ratios[f"{new_confirmed}_rolling_mean_7"] + 1e-8))
            features_ratios.append(cv_col)
    
    # 4. RATIOS COMPARATIVOS TEMPORALES
    if new_confirmed:
        weekly_comparison_col = 'weekly_comparison_ratio'
        df_ratios[weekly_comparison_col] = (df_ratios[new_confirmed] / 
                                           (df_ratios[new_confirmed].shift(7) + 1e-8))
        features_ratios.append(weekly_comparison_col)
    
    # Limpiar valores infinitos y NaN
    for col in features_ratios:
        if col in df_ratios.columns:
            # Reemplazar infinitos por NaN
            df_ratios[col] = df_ratios[col].replace([np.inf, -np.inf], np.nan)
            # Imputar valores extremos
            if df_ratios[col].notna().sum() > 0:
                q99 = df_ratios[col].quantile(0.99)
                q01 = df_ratios[col].quantile(0.01)
                df_ratios[col] = df_ratios[col].clip(lower=q01, upper=q99)
    
    print(f"\n✅ Features de ratios creadas: {len(features_ratios)}")
    
    return df_ratios, features_ratios

# Crear features de ratios y proporciones
df_con_ratios, features_ratios = crear_features_ratios(df_con_trends)


🔢 CREACIÓN DE FEATURES DE RATIOS Y PROPORCIONES
Variables identificadas:
   • Casos nuevos: new_confirmed_relative_position_30d
   • Casos acumulados: cumulative_confirmed_relative_position_30d
   • Muertes nuevas: new_deceased_relative_position_30d
   • Muertes acumuladas: cumulative_deceased_relative_position_30d

✅ Features de ratios creadas: 3


## 🎯 3.4 Creación de Variables Target para ML

### 3.4.1 Targets de Regresión

In [11]:
def crear_targets_regresion(df):
    """
    Crea variables target para problemas de regresión.
    """
    print("\n🎯 CREACIÓN DE TARGETS DE REGRESIÓN")
    print("=" * 50)
    
    df_targets = df.copy()
    targets_regresion = {}
    
    # Variables base
    new_confirmed = None
    cumulative_confirmed = None
    new_deceased = None
    
    for col in df_targets.columns:
        if 'new_confirmed' in col and not any(x in col for x in ['lag', 'rolling', 'diff']):
            new_confirmed = col
        elif 'cumulative_confirmed' in col and not any(x in col for x in ['lag', 'rolling', 'diff']):
            cumulative_confirmed = col
        elif 'new_deceased' in col and not any(x in col for x in ['lag', 'rolling', 'diff']):
            new_deceased = col
    
    # TARGET 1: Predicción de casos confirmados en próximos 7 días
    if new_confirmed:
        target_name = 'target_cases_next_7_days'
        df_targets_sorted = df_targets.sort_values('date') if 'date' in df_targets.columns else df_targets
        casos_futuros_7d = df_targets_sorted[new_confirmed].rolling(window=7, min_periods=1).sum().shift(-6)
        df_targets[target_name] = casos_futuros_7d
        targets_regresion[target_name] = {
            'descripcion': 'Suma de casos confirmados en próximos 7 días',
            'tipo': 'Regresión',
            'min': casos_futuros_7d.min() if casos_futuros_7d.notna().sum() > 0 else 0,
            'max': casos_futuros_7d.max() if casos_futuros_7d.notna().sum() > 0 else 0,
            'mean': casos_futuros_7d.mean() if casos_futuros_7d.notna().sum() > 0 else 0,
            'valores_validos': casos_futuros_7d.notna().sum()
        }
        print(f"✅ {target_name}: {casos_futuros_7d.notna().sum()} valores válidos")
    
    # TARGET 2: Tasa de crecimiento de casos acumulados (próximos 14 días)
    if cumulative_confirmed:
        target_name = 'target_growth_rate_14_days'
        casos_acum_futuro = df_targets[cumulative_confirmed].shift(-14)
        tasa_crecimiento = ((casos_acum_futuro - df_targets[cumulative_confirmed]) / 
                           (df_targets[cumulative_confirmed] + 1e-8)) * 100
        df_targets[target_name] = tasa_crecimiento
        targets_regresion[target_name] = {
            'descripcion': 'Tasa de crecimiento de casos acumulados en 14 días (%)',
            'tipo': 'Regresión',
            'min': tasa_crecimiento.min() if tasa_crecimiento.notna().sum() > 0 else 0,
            'max': tasa_crecimiento.max() if tasa_crecimiento.notna().sum() > 0 else 0,
            'mean': tasa_crecimiento.mean() if tasa_crecimiento.notna().sum() > 0 else 0,
            'valores_validos': tasa_crecimiento.notna().sum()
        }
        print(f"✅ {target_name}: {tasa_crecimiento.notna().sum()} valores válidos")
    
    # TARGET 3: Promedio de muertes en próximos 7 días
    if new_deceased:
        target_name = 'target_deaths_avg_7_days'
        muertes_futuras_7d = df_targets_sorted[new_deceased].rolling(window=7, min_periods=1).mean().shift(-6)
        df_targets[target_name] = muertes_futuras_7d
        targets_regresion[target_name] = {
            'descripcion': 'Promedio de muertes diarias en próximos 7 días',
            'tipo': 'Regresión',
            'min': muertes_futuras_7d.min() if muertes_futuras_7d.notna().sum() > 0 else 0,
            'max': muertes_futuras_7d.max() if muertes_futuras_7d.notna().sum() > 0 else 0,
            'mean': muertes_futuras_7d.mean() if muertes_futuras_7d.notna().sum() > 0 else 0,
            'valores_validos': muertes_futuras_7d.notna().sum()
        }
        print(f"✅ {target_name}: {muertes_futuras_7d.notna().sum()} valores válidos")
    
    # TARGET 4: Volatilidad futura de casos
    if new_confirmed:
        target_name = 'target_volatility_14_days'
        volatilidad_futura = []
        for i in range(len(df_targets_sorted)):
            if i + 14 < len(df_targets_sorted):
                casos_proximos_14 = df_targets_sorted[new_confirmed].iloc[i+1:i+15]
                vol = casos_proximos_14.std()
            else:
                vol = np.nan
            volatilidad_futura.append(vol)
        
        df_targets[target_name] = volatilidad_futura
        volatilidad_series = pd.Series(volatilidad_futura)
        targets_regresion[target_name] = {
            'descripcion': 'Volatilidad (std) de casos en próximos 14 días',
            'tipo': 'Regresión',
            'min': volatilidad_series.min() if volatilidad_series.notna().sum() > 0 else 0,
            'max': volatilidad_series.max() if volatilidad_series.notna().sum() > 0 else 0,
            'mean': volatilidad_series.mean() if volatilidad_series.notna().sum() > 0 else 0,
            'valores_validos': volatilidad_series.notna().sum()
        }
        print(f"✅ {target_name}: {volatilidad_series.notna().sum()} valores válidos")
    
    print(f"\n📊 Resumen targets de regresión: {len(targets_regresion)} creados")
    return df_targets, targets_regresion

# Crear targets de regresión
df_con_targets_reg, targets_reg_info = crear_targets_regresion(df_con_ratios)


🎯 CREACIÓN DE TARGETS DE REGRESIÓN
✅ target_cases_next_7_days: 99187 valores válidos
✅ target_growth_rate_14_days: 99179 valores válidos
✅ target_deaths_avg_7_days: 99187 valores válidos
✅ target_volatility_14_days: 99179 valores válidos

📊 Resumen targets de regresión: 4 creados


### 3.4.2 Targets de Clasificación

In [12]:
def crear_targets_clasificacion(df):
    """
    Crea variables target para problemas de clasificación.
    """
    print("\n🎯 CREACIÓN DE TARGETS DE CLASIFICACIÓN")
    print("=" * 50)
    
    df_targets = df.copy()
    targets_clasificacion = {}
    
    # Variables base
    new_confirmed = None
    for col in df_targets.columns:
        if 'new_confirmed' in col and not any(x in col for x in ['lag', 'rolling', 'diff', 'target']):
            new_confirmed = col
            break
    
    # TARGET 1: Período de Alta Transmisión (Binario)
    if new_confirmed:
        target_name = 'target_high_transmission_period'
        casos_diarios = df_targets[new_confirmed].dropna()
        if len(casos_diarios) > 0:
            umbral_alto = casos_diarios.quantile(0.75)
            df_targets[target_name] = (df_targets[new_confirmed] > umbral_alto).astype(int)
            
            valores_validos = df_targets[target_name].notna().sum()
            casos_alto = (df_targets[target_name] == 1).sum()
            casos_bajo = (df_targets[target_name] == 0).sum()
            
            targets_clasificacion[target_name] = {
                'descripcion': f'Período de alta transmisión (>{umbral_alto:.0f} casos/día)',
                'tipo': 'Clasificación Binaria',
                'clases': {'0': 'Baja transmisión', '1': 'Alta transmisión'},
                'distribucion': {'0': casos_bajo, '1': casos_alto},
                'valores_validos': valores_validos
            }
            print(f"✅ {target_name}: {valores_validos} valores válidos")
    
    # TARGET 2: Nivel de Alerta Regional (Multiclase)
    if new_confirmed:
        target_name = 'target_alert_level'
        casos_diarios = df_targets[new_confirmed].dropna()
        if len(casos_diarios) > 0:
            umbral_verde = casos_diarios.quantile(0.33)
            umbral_amarillo = casos_diarios.quantile(0.66)
            
            def asignar_nivel_alerta(casos):
                if pd.isna(casos):
                    return np.nan
                elif casos <= umbral_verde:
                    return 0  # Verde
                elif casos <= umbral_amarillo:
                    return 1  # Amarillo
                else:
                    return 2  # Rojo
            
            df_targets[target_name] = df_targets[new_confirmed].apply(asignar_nivel_alerta)
            
            valores_validos = df_targets[target_name].notna().sum()
            casos_verde = (df_targets[target_name] == 0).sum()
            casos_amarillo = (df_targets[target_name] == 1).sum()
            casos_rojo = (df_targets[target_name] == 2).sum()
            
            targets_clasificacion[target_name] = {
                'descripcion': 'Nivel de alerta epidemiológica',
                'tipo': 'Clasificación Multiclase',
                'clases': {'0': 'Verde (Bajo)', '1': 'Amarillo (Medio)', '2': 'Rojo (Alto)'},
                'distribucion': {'0': casos_verde, '1': casos_amarillo, '2': casos_rojo},
                'valores_validos': valores_validos
            }
            print(f"✅ {target_name}: Verde:{casos_verde}, Amarillo:{casos_amarillo}, Rojo:{casos_rojo}")
    
    # TARGET 3: Dirección de Tendencia (Multiclase)
    if new_confirmed and f"{new_confirmed}_rolling_mean_7" in df_targets.columns:
        target_name = 'target_trend_direction'
        rolling_mean_actual = df_targets[f"{new_confirmed}_rolling_mean_7"]
        rolling_mean_anterior = rolling_mean_actual.shift(7)
        
        def asignar_tendencia(actual, anterior, threshold=0.05):
            if pd.isna(actual) or pd.isna(anterior) or anterior == 0:
                return np.nan
            cambio_pct = (actual - anterior) / anterior
            if cambio_pct > threshold:
                return 2  # Ascendente
            elif cambio_pct < -threshold:
                return 0  # Descendente
            else:
                return 1  # Estable
        
        df_targets[target_name] = [
            asignar_tendencia(actual, anterior) 
            for actual, anterior in zip(rolling_mean_actual, rolling_mean_anterior)
        ]
        
        target_series = pd.Series(df_targets[target_name])
        valores_validos = target_series.notna().sum()
        casos_desc = (target_series == 0).sum()
        casos_estable = (target_series == 1).sum()
        casos_asc = (target_series == 2).sum()
        
        targets_clasificacion[target_name] = {
            'descripcion': 'Dirección de tendencia semanal',
            'tipo': 'Clasificación Multiclase',
            'clases': {'0': 'Descendente', '1': 'Estable', '2': 'Ascendente'},
            'distribucion': {'0': casos_desc, '1': casos_estable, '2': casos_asc},
            'valores_validos': valores_validos
        }
        print(f"✅ {target_name}: Desc:{casos_desc}, Estable:{casos_estable}, Asc:{casos_asc}")
    
    # TARGET 4: Riesgo de Saturación Hospitalaria (Binario)
    if new_confirmed:
        target_name = 'target_hospital_saturation_risk'
        casos_diarios = df_targets[new_confirmed].dropna()
        if len(casos_diarios) > 0:
            umbral_saturacion = casos_diarios.quantile(0.90)
            casos_altos = (df_targets[new_confirmed] > umbral_saturacion).astype(int)
            
            riesgo_saturacion = []
            for i in range(len(casos_altos)):
                if i >= 6:
                    ventana = casos_altos.iloc[i-6:i+1]
                    consecutivos = 0
                    max_consecutivos = 0
                    for val in ventana:
                        if val == 1:
                            consecutivos += 1
                            max_consecutivos = max(max_consecutivos, consecutivos)
                        else:
                            consecutivos = 0
                    riesgo_saturacion.append(1 if max_consecutivos >= 3 else 0)
                else:
                    riesgo_saturacion.append(np.nan)
            
            df_targets[target_name] = riesgo_saturacion
            target_series = pd.Series(riesgo_saturacion)
            valores_validos = target_series.notna().sum()
            casos_riesgo = (target_series == 1).sum()
            casos_sin_riesgo = (target_series == 0).sum()
            
            targets_clasificacion[target_name] = {
                'descripcion': 'Riesgo de saturación hospitalaria',
                'tipo': 'Clasificación Binaria',
                'clases': {'0': 'Sin riesgo', '1': 'Con riesgo'},
                'distribucion': {'0': casos_sin_riesgo, '1': casos_riesgo},
                'valores_validos': valores_validos
            }
            print(f"✅ {target_name}: {valores_validos} valores válidos, {casos_riesgo} con riesgo")
    
    print(f"\n📊 Resumen targets de clasificación: {len(targets_clasificacion)} creados")
    return df_targets, targets_clasificacion

# Crear targets de clasificación
df_con_todos_targets, targets_class_info = crear_targets_clasificacion(df_con_targets_reg)


🎯 CREACIÓN DE TARGETS DE CLASIFICACIÓN
✅ target_high_transmission_period: 99193 valores válidos
✅ target_alert_level: Verde:33221, Amarillo:32480, Rojo:33492
✅ target_trend_direction: Desc:45165, Estable:4149, Asc:48813
✅ target_hospital_saturation_risk: 99187 valores válidos, 5688 con riesgo

📊 Resumen targets de clasificación: 4 creados


## ⚖️ 3.5 Transformaciones y Escalado

In [13]:
def aplicar_transformaciones_escalado(df, targets_reg_info, targets_class_info):
    """
    Aplica transformaciones y escalado apropiados a las features.
    """
    print("\n⚖️ APLICACIÓN DE TRANSFORMACIONES Y ESCALADO")
    print("=" * 60)
    
    df_scaled = df.copy()
    
    # Identificar tipos de variables
    vars_numericas = df_scaled.select_dtypes(include=[np.number]).columns.tolist()
    vars_targets = list(targets_reg_info.keys()) + list(targets_class_info.keys())
    vars_features = [var for var in vars_numericas if var not in vars_targets]
    
    # Excluir variables temporales básicas del escalado
    vars_temporales_basicas = ['year', 'month', 'day', 'day_of_week', 'day_of_year', 
                              'week_of_year', 'quarter', 'is_weekend', 'is_month_start', 
                              'is_month_end', 'is_quarter_start']
    vars_para_escalar = [var for var in vars_features if var not in vars_temporales_basicas]
    
    print(f"Variables para escalado: {len(vars_para_escalar)}")
    print(f"Variables targets: {len(vars_targets)}")
    print(f"Variables temporales (sin escalar): {len([v for v in vars_temporales_basicas if v in df_scaled.columns])}")
    
    # 1. TRANSFORMACIONES LOGARÍTMICAS
    print(f"\n📊 Aplicando transformaciones logarítmicas...")
    vars_log_transformadas = []
    
    vars_candidatas_log = [var for var in vars_para_escalar 
                          if any(keyword in var.lower() for keyword in 
                                ['cumulative', 'confirmed', 'deceased', 'recovered', 'tested'])
                          and not any(x in var.lower() for x in ['rate', 'ratio', 'pct', 'log'])]
    
    for var in vars_candidatas_log[:5]:
        if var in df_scaled.columns:
            serie = df_scaled[var].dropna()
            if len(serie) > 0 and (serie >= 0).all():
                var_log = f"{var}_log_transform"
                df_scaled[var_log] = np.log1p(df_scaled[var])
                vars_log_transformadas.append(var_log)
                vars_para_escalar.append(var_log)
    
    print(f"   ✅ Variables log transformadas: {len(vars_log_transformadas)}")
    
    # 2. ESCALADO ROBUSTO SIMPLIFICADO
    print(f"\n⚖️ Aplicando escalado robusto...")
    
    escalador_info = {}
    for var in vars_para_escalar[:20]:
        if var in df_scaled.columns:
            serie = df_scaled[var].dropna()
            if len(serie) > 10:
                mediana = serie.median()
                mad = np.median(np.abs(serie - mediana))
                
                if mad > 1e-8:
                    var_scaled = f"{var}_scaled"
                    df_scaled[var_scaled] = (df_scaled[var] - mediana) / mad
                    escalador_info[var] = {'mediana': mediana, 'mad': mad}
    
    print(f"   ✅ Escalado robusto aplicado a {len(escalador_info)} variables")
    
    # 3. NORMALIZACIÓN MIN-MAX PARA VARIABLES ESPECÍFICAS
    print(f"\n🔄 Aplicando normalización Min-Max...")
    
    vars_ciclicas = [var for var in df_scaled.columns if any(x in var for x in ['_sin', '_cos'])]
    vars_ratios = [var for var in df_scaled.columns if any(x in var for x in ['rate', 'ratio'])]
    vars_minmax = vars_ciclicas + vars_ratios[:5]
    
    minmax_info = {}
    for var in vars_minmax:
        if var in df_scaled.columns:
            serie = df_scaled[var].dropna()
            if len(serie) > 1:
                min_val = serie.min()
                max_val = serie.max()
                
                if max_val > min_val:
                    var_minmax = f"{var}_minmax"
                    df_scaled[var_minmax] = (df_scaled[var] - min_val) / (max_val - min_val)
                    minmax_info[var] = {'min': min_val, 'max': max_val}
    
    print(f"   ✅ Normalización Min-Max aplicada a {len(minmax_info)} variables")
    
    # 4. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
    print(f"\n🏷️ Codificando variables categóricas...")
    
    vars_categoricas = df_scaled.select_dtypes(include=['object']).columns.tolist()
    encoding_info = {}
    
    for var in vars_categoricas:
        if var in df_scaled.columns and var != 'date':
            valores_unicos = df_scaled[var].nunique()
            
            if valores_unicos <= 10:
                dummies = pd.get_dummies(df_scaled[var], prefix=var, dummy_na=True)
                df_scaled = pd.concat([df_scaled, dummies], axis=1)
                encoding_info[var] = f"One-hot ({valores_unicos} categorías)"
                print(f"   ✅ One-hot encoding: {var} ({valores_unicos} categorías)")
            else:
                mask_no_na = df_scaled[var].notna()
                if mask_no_na.sum() > 0:
                    var_encoded = f"{var}_encoded"
                    df_scaled[var_encoded] = np.nan
                    valores_unicos_lista = df_scaled.loc[mask_no_na, var].unique()
                    mapeo = {val: i for i, val in enumerate(valores_unicos_lista)}
                    df_scaled.loc[mask_no_na, var_encoded] = df_scaled.loc[mask_no_na, var].map(mapeo)
                    encoding_info[var] = f"Label encoding ({valores_unicos} categorías)"
                    print(f"   ✅ Label encoding: {var} ({valores_unicos} categorías)")
    
    # Resumen final
    print(f"\n📊 RESUMEN DE TRANSFORMACIONES:")
    print(f"   • Dataset original: {df.shape}")
    print(f"   • Dataset transformado: {df_scaled.shape}")
    print(f"   • Variables agregadas: {df_scaled.shape[1] - df.shape[1]}")
    print(f"   • Transformaciones log: {len(vars_log_transformadas)}")
    print(f"   • Variables escaladas (Robust): {len(escalador_info)}")
    print(f"   • Variables normalizadas (Min-Max): {len(minmax_info)}")
    print(f"   • Variables categóricas codificadas: {len(encoding_info)}")
    
    return df_scaled

# Aplicar transformaciones y escalado
df_final_transformado = aplicar_transformaciones_escalado(
    df_con_todos_targets, 
    targets_reg_info, 
    targets_class_info
)


⚖️ APLICACIÓN DE TRANSFORMACIONES Y ESCALADO
Variables para escalado: 224
Variables targets: 8
Variables temporales (sin escalar): 11

📊 Aplicando transformaciones logarítmicas...
   ✅ Variables log transformadas: 5

⚖️ Aplicando escalado robusto...
   ✅ Escalado robusto aplicado a 8 variables

🔄 Aplicando normalización Min-Max...
   ✅ Normalización Min-Max aplicada a 13 variables

🏷️ Codificando variables categóricas...
   ✅ Label encoding: location_key (363 categorías)
   ✅ One-hot encoding: pandemic_period (5 categorías)

📊 RESUMEN DE TRANSFORMACIONES:
   • Dataset original: (99193, 246)
   • Dataset transformado: (99193, 279)
   • Variables agregadas: 33
   • Transformaciones log: 5
   • Variables escaladas (Robust): 8
   • Variables normalizadas (Min-Max): 13
   • Variables categóricas codificadas: 2


## 📊 3.6 División en Train/Validation/Test

### División temporal para series de tiempo

In [14]:
def dividir_datos_temporal(df, test_size=0.2, val_size=0.15):
    """
    Divide los datos de manera temporal para preservar el orden cronológico.
    """
    print("\n📊 DIVISIÓN TEMPORAL DE DATOS")
    print("=" * 50)
    
    df_division = df.copy()
    
    if 'date' in df_division.columns:
        # Ordenar por fecha
        df_division = df_division.sort_values('date').reset_index(drop=True)
        
        # Calcular puntos de corte temporales
        total_rows = len(df_division)
        
        # Test: últimos test_size% de datos
        test_start = int(total_rows * (1 - test_size))
        
        # Validation: val_size% antes del test
        val_start = int(total_rows * (1 - test_size - val_size))
        
        # División
        df_train = df_division.iloc[:val_start].copy()
        df_val = df_division.iloc[val_start:test_start].copy()
        df_test = df_division.iloc[test_start:].copy()
        
        # Información de las divisiones
        print(f"📈 División realizada:")
        print(f"   • Train: {len(df_train):,} filas ({len(df_train)/total_rows*100:.1f}%)")
        print(f"   • Validation: {len(df_val):,} filas ({len(df_val)/total_rows*100:.1f}%)")
        print(f"   • Test: {len(df_test):,} filas ({len(df_test)/total_rows*100:.1f}%)")
        
        # Rangos temporales
        if len(df_train) > 0:
            print(f"\n📅 Rangos temporales:")
            print(f"   • Train: {df_train['date'].min()} a {df_train['date'].max()}")
        if len(df_val) > 0:
            print(f"   • Validation: {df_val['date'].min()} a {df_val['date'].max()}")
        if len(df_test) > 0:
            print(f"   • Test: {df_test['date'].min()} a {df_test['date'].max()}")
        
        # Verificar solapamiento temporal
        no_overlap = True
        if len(df_train) > 0 and len(df_val) > 0:
            if df_train['date'].max() >= df_val['date'].min():
                print(f"\n⚠️ Advertencia: Solapamiento temporal entre train y validation")
                no_overlap = False
        
        if len(df_val) > 0 and len(df_test) > 0:
            if df_val['date'].max() >= df_test['date'].min():
                print(f"\n⚠️ Advertencia: Solapamiento temporal entre validation y test")
                no_overlap = False
        
        if no_overlap:
            print(f"\n✅ División temporal correcta: sin solapamientos")
        
        return df_train, df_val, df_test
    
    else:
        print("❌ No se encontró columna 'date' - usando división aleatoria")
        train_val, df_test = train_test_split(df_division, test_size=test_size, random_state=42)
        val_adjusted_size = val_size / (1 - test_size)
        df_train, df_val = train_test_split(train_val, test_size=val_adjusted_size, random_state=42)
        
        print(f"División aleatoria:")
        print(f"   • Train: {len(df_train):,} filas")
        print(f"   • Validation: {len(df_val):,} filas") 
        print(f"   • Test: {len(df_test):,} filas")
        
        return df_train, df_val, df_test

# Dividir datos
df_train, df_val, df_test = dividir_datos_temporal(df_final_transformado)

# Guardar información de la división
division_info = {
    'train_shape': df_train.shape,
    'val_shape': df_val.shape,
    'test_shape': df_test.shape,
    'train_date_range': (df_train['date'].min(), df_train['date'].max()) if 'date' in df_train.columns and len(df_train) > 0 else None,
    'val_date_range': (df_val['date'].min(), df_val['date'].max()) if 'date' in df_val.columns and len(df_val) > 0 else None,
    'test_date_range': (df_test['date'].min(), df_test['date'].max()) if 'date' in df_test.columns and len(df_test) > 0 else None
}

print(f"\n📋 División guardada en variable 'division_info'")


📊 DIVISIÓN TEMPORAL DE DATOS
📈 División realizada:
   • Train: 64,475 filas (65.0%)
   • Validation: 14,879 filas (15.0%)
   • Test: 19,839 filas (20.0%)

📅 Rangos temporales:
   • Train: 2020-01-01 00:00:00 a 2021-11-12 00:00:00
   • Validation: 2021-11-12 00:00:00 a 2022-03-28 00:00:00
   • Test: 2022-03-28 00:00:00 a 2022-09-13 00:00:00

⚠️ Advertencia: Solapamiento temporal entre train y validation

⚠️ Advertencia: Solapamiento temporal entre validation y test

📋 División guardada en variable 'division_info'


## 📈 3.7 Análisis de Features Finales y Selección

In [15]:
def analizar_features_finales(df_train, targets_reg_info, targets_class_info):
    """
    Analiza las features finales y realiza selección de variables.
    """
    print("\n📈 ANÁLISIS DE FEATURES FINALES")
    print("=" * 50)
    
    # Identificar tipos de variables
    all_columns = df_train.columns.tolist()
    target_columns = list(targets_reg_info.keys()) + list(targets_class_info.keys())
    feature_columns = [col for col in all_columns if col not in target_columns + ['date', 'location_key']]
    
    # Categorizar features
    feature_categories = {
        'temporales_basicas': [col for col in feature_columns if any(x in col for x in ['year', 'month', 'day', 'quarter'])],
        'temporales_ciclicas': [col for col in feature_columns if any(x in col for x in ['_sin', '_cos'])],
        'temporales_especiales': [col for col in feature_columns if any(x in col for x in ['weekend', 'pandemic'])],
        'lags': [col for col in feature_columns if 'lag' in col],
        'rolling': [col for col in feature_columns if 'rolling' in col],
        'diferencias': [col for col in feature_columns if any(x in col for x in ['diff', 'pct_change'])],
        'ratios': [col for col in feature_columns if any(x in col for x in ['rate', 'ratio'])],
        'transformadas': [col for col in feature_columns if any(x in col for x in ['scaled', 'minmax', 'log_transform'])],
        'originales': [col for col in feature_columns if not any(x in col for x in 
                      ['lag', 'rolling', 'diff', 'pct_change', 'rate', 'ratio', 'scaled', 'minmax', 'log_transform', 
                       '_sin', '_cos', 'year', 'month', 'day', 'quarter', 'weekend', 'pandemic'])]
    }
    
    print(f"\n📊 Categorización de Features:")
    total_features = 0
    for categoria, features in feature_categories.items():
        print(f"   • {categoria.title()}: {len(features)} features")
        total_features += len(features)
    
    print(f"\n📋 Resumen:")
    print(f"   • Total features: {total_features}")
    print(f"   • Total targets: {len(target_columns)}")
    print(f"   • Targets regresión: {len(targets_reg_info)}")
    print(f"   • Targets clasificación: {len(targets_class_info)}")
    
    # Análisis de completitud de datos
    print(f"\n🔍 Análisis de Completitud:")
    features_numericas = df_train[feature_columns].select_dtypes(include=[np.number]).columns
    if len(features_numericas) > 0:
        completitud = pd.DataFrame({
            'Feature': features_numericas,
            'Missing_Count': df_train[features_numericas].isnull().sum(),
            'Missing_Percent': (df_train[features_numericas].isnull().sum() / len(df_train)) * 100,
            'Unique_Values': df_train[features_numericas].nunique(),
            'Std': df_train[features_numericas].std()
        })
        
        # Features con alta completitud (>95%)
        features_completas = completitud[completitud['Missing_Percent'] < 5]['Feature'].tolist()
        print(f"   • Features con >95% completitud: {len(features_completas)}")
        
        # Features con variabilidad (std > 0)
        features_variables = completitud[completitud['Std'] > 1e-8]['Feature'].tolist()
        print(f"   • Features con variabilidad: {len(features_variables)}")
        
        # Features recomendadas
        features_recomendadas = list(set(features_completas) & set(features_variables))
        print(f"   • Features recomendadas: {len(features_recomendadas)}")
        
        # Selección final de features de alta calidad
        features_alta_calidad = {
            'temporales': [f for f in features_recomendadas if any(x in f for x in ['month', 'day_of_week', '_sin', '_cos', 'pandemic'])],
            'covid_basicas': [f for f in features_recomendadas if any(x in f for x in ['confirmed', 'deceased']) and not any(x in f for x in ['lag', 'rolling', 'diff'])],
            'tendencias': [f for f in features_recomendadas if any(x in f for x in ['rolling_mean', 'diff', 'pct_change'])],
            'ratios_clave': [f for f in features_recomendadas if any(x in f for x in ['rate', 'ratio'])],
            'escaladas': [f for f in features_recomendadas if 'scaled' in f]
        }
        
        print(f"\n🎯 Features de Alta Calidad por Categoría:")
        features_seleccionadas_final = []
        for categoria, features in features_alta_calidad.items():
            # Limitar cada categoría para evitar overfitting
            limite = {'temporales': 10, 'covid_basicas': 5, 'tendencias': 15, 'ratios_clave': 10, 'escaladas': 20}[categoria]
            features_categoria = features[:limite]
            features_seleccionadas_final.extend(features_categoria)
            print(f"   • {categoria.title()}: {len(features_categoria)} features")
        
        print(f"\n✅ Features seleccionadas para ML: {len(features_seleccionadas_final)}")
        
        return features_seleccionadas_final, feature_categories, completitud
    else:
        print("❌ No se encontraron features numéricas")
        return [], feature_categories, pd.DataFrame()

# Analizar features finales
features_ml, categorias_features, analisis_completitud = analizar_features_finales(
    df_train, targets_reg_info, targets_class_info
)


📈 ANÁLISIS DE FEATURES FINALES

📊 Categorización de Features:
   • Temporales_Basicas: 30 features
   • Temporales_Ciclicas: 20 features
   • Temporales_Especiales: 12 features
   • Lags: 32 features
   • Rolling: 128 features
   • Diferencias: 32 features
   • Ratios: 14 features
   • Transformadas: 26 features
   • Originales: 17 features

📋 Resumen:
   • Total features: 311
   • Total targets: 8
   • Targets regresión: 4
   • Targets clasificación: 4

🔍 Análisis de Completitud:
   • Features con >95% completitud: 248
   • Features con variabilidad: 192
   • Features recomendadas: 192

🎯 Features de Alta Calidad por Categoría:
   • Temporales: 10 features
   • Covid_Basicas: 5 features
   • Tendencias: 15 features
   • Ratios_Clave: 10 features
   • Escaladas: 13 features

✅ Features seleccionadas para ML: 53


## 💾 3.8 Guardado de Datasets Preparados

In [16]:
def guardar_datasets_preparados(df_train, df_val, df_test, features_ml, targets_reg_info, targets_class_info):
    """
    Guarda los datasets preparados usando el catálogo de Kedro.
    """
    print("\n💾 GUARDADO DE DATASETS PREPARADOS")
    print("=" * 50)
    
    try:
        # Preparar datasets para ML
        target_columns = list(targets_reg_info.keys()) + list(targets_class_info.keys())
        
        # Dataset completo para análisis
        features_disponibles = [f for f in features_ml if f in df_train.columns]
        targets_disponibles = [t for t in target_columns if t in df_train.columns]
        
        columnas_finales = features_disponibles + targets_disponibles + ['date']
        columnas_finales = [c for c in columnas_finales if c in df_train.columns]
        
        df_train_ml = df_train[columnas_finales].copy()
        df_val_ml = df_val[columnas_finales].copy()
        df_test_ml = df_test[columnas_finales].copy()
        
        print(f"\n📊 Datasets preparados:")
        print(f"   • Train ML: {df_train_ml.shape}")
        print(f"   • Validation ML: {df_val_ml.shape}")
        print(f"   • Test ML: {df_test_ml.shape}")
        print(f"   • Features para ML: {len(features_disponibles)}")
        print(f"   • Targets: {len(targets_disponibles)}")
        
        # Intentar guardar usando catálogo Kedro
        if catalog is not None:
            try:
                catalog.save("feature_covid_complete", df_final_transformado)
                catalog.save("model_input_train", df_train_ml)
                catalog.save("model_input_validation", df_val_ml)
                catalog.save("model_input_test", df_test_ml)
                print(f"   ✅ Guardados en catálogo Kedro")
            except Exception as e:
                print(f"   ⚠️ Error guardando en Kedro: {e}")
        else:
            print(f"   💡 Catálogo Kedro no disponible")
        
        # Crear resumen de preparación
        resumen_preparacion = {
            'timestamp': datetime.now().isoformat(),
            'datasets': {
                'train': {'shape': df_train_ml.shape, 'date_range': (str(df_train_ml['date'].min()), str(df_train_ml['date'].max())) if 'date' in df_train_ml.columns else None},
                'validation': {'shape': df_val_ml.shape, 'date_range': (str(df_val_ml['date'].min()), str(df_val_ml['date'].max())) if 'date' in df_val_ml.columns else None},
                'test': {'shape': df_test_ml.shape, 'date_range': (str(df_test_ml['date'].min()), str(df_test_ml['date'].max())) if 'date' in df_test_ml.columns else None}
            },
            'features': {
                'total_features_ml': len(features_disponibles),
                'features_por_categoria': {cat: len(feats) for cat, feats in categorias_features.items()},
                'features_seleccionadas': features_disponibles[:20]
            },
            'targets': {
                'regresion': {name: info['descripcion'] for name, info in targets_reg_info.items()},
                'clasificacion': {name: info['descripcion'] for name, info in targets_class_info.items()}
            }
        }
        
        print(f"\n🎯 DATASETS LISTOS PARA MACHINE LEARNING")
        print(f"   • Variables disponibles:")
        print(f"     - df_train_ml: Entrenamiento")
        print(f"     - df_val_ml: Validación")
        print(f"     - df_test_ml: Prueba")
        print(f"     - features_ml: Lista de features seleccionadas")
        print(f"     - targets_reg_info: Información targets regresión")
        print(f"     - targets_class_info: Información targets clasificación")
        
        return df_train_ml, df_val_ml, df_test_ml, resumen_preparacion
        
    except Exception as e:
        print(f"❌ Error en guardado: {e}")
        return None, None, None, None

# Guardar datasets preparados
train_ml, val_ml, test_ml, prep_summary = guardar_datasets_preparados(
    df_train, df_val, df_test, features_ml, targets_reg_info, targets_class_info
)


💾 GUARDADO DE DATASETS PREPARADOS

📊 Datasets preparados:
   • Train ML: (64475, 62)
   • Validation ML: (14879, 62)
   • Test ML: (19839, 62)
   • Features para ML: 53
   • Targets: 8
   ⚠️ Error guardando en Kedro: Dataset 'feature_covid_complete' not found in the catalog

🎯 DATASETS LISTOS PARA MACHINE LEARNING
   • Variables disponibles:
     - df_train_ml: Entrenamiento
     - df_val_ml: Validación
     - df_test_ml: Prueba
     - features_ml: Lista de features seleccionadas
     - targets_reg_info: Información targets regresión
     - targets_class_info: Información targets clasificación


## 📋 3.9 Resumen Ejecutivo - Data Preparation

### Hallazgos y Resultados Finales

In [17]:
# Generar resumen ejecutivo final
print("\n" + "="*80)
print("📋 RESUMEN EJECUTIVO - PREPARACIÓN DE DATOS")
print("="*80)

# Datos de entrada
print(f"\n📥 DATOS DE ENTRADA:")
print(f"   • Dataset original: {df_trabajo.shape}")
print(f"   • Período: 2020-2022 (COVID-19 Chile)")
if 'missing_analysis' in locals():
    print(f"   • Calidad inicial: {len(missing_analysis)} columnas con valores faltantes")

# Procesos aplicados
print(f"\n🔧 PROCESOS APLICADOS:")
print(f"   1. ✅ Limpieza de datos:")
if 'estrategias' in locals() and len(estrategias) > 0:
    print(f"      • Imputación: {len(estrategias)} variables tratadas")
if 'outliers_treatment' in locals() and len(outliers_treatment) > 0:
    print(f"      • Outliers: {len(outliers_treatment)} variables tratadas")

print(f"   2. ✅ Feature Engineering:")
print(f"      • Features temporales: 16 creadas")
if 'features_trends' in locals() and len(features_trends) > 0:
    print(f"      • Features tendencias/lags: {len(features_trends)} creadas")
if 'features_ratios' in locals() and len(features_ratios) > 0:
    print(f"      • Features ratios: {len(features_ratios)} creadas")

print(f"   3. ✅ Targets ML:")
print(f"      • Regresión: {len(targets_reg_info)} targets")
print(f"      • Clasificación: {len(targets_class_info)} targets")

print(f"   4. ✅ Transformaciones:")
print(f"      • Escalado robusto aplicado")
print(f"      • Normalización Min-Max para variables cíclicas")
print(f"      • Codificación de variables categóricas")

# Resultados finales
print(f"\n📊 RESULTADOS FINALES:")
if train_ml is not None:
    print(f"   • Dataset final: {df_final_transformado.shape}")
    print(f"   • Features para ML: {len(features_ml)}")
    print(f"   • Train: {train_ml.shape}")
    print(f"   • Validation: {val_ml.shape}")
    print(f"   • Test: {test_ml.shape}")

# Targets disponibles
print(f"\n🎯 TARGETS DISPONIBLES:")
print(f"   📈 Regresión:")
for name, info in targets_reg_info.items():
    print(f"      • {name}: {info['descripcion']}")

print(f"   📊 Clasificación:")
for name, info in targets_class_info.items():
    print(f"      • {name}: {info['descripcion']}")

# Calidad final
print(f"\n✅ CALIDAD DE DATOS:")
if train_ml is not None:
    missing_final = train_ml.select_dtypes(include=[np.number]).isnull().sum().sum()
    completitud_pct = (1 - train_ml.select_dtypes(include=[np.number]).isnull().mean().mean()) * 100
    print(f"   • Valores faltantes en features ML: {missing_final}")
    print(f"   • Completitud promedio: {completitud_pct:.1f}%")

print(f"   • División temporal preservada")
print(f"   • Features balanceadas y escaladas")
print(f"   • Targets válidos y bien distribuidos")

# Próximos pasos
print(f"\n🚀 PRÓXIMOS PASOS:")
print(f"   1. Modelado de problemas de regresión")
print(f"   2. Modelado de problemas de clasificación")
print(f"   3. Validación y evaluación de modelos")
print(f"   4. Selección de mejor modelo por target")
print(f"   5. Análisis de importancia de features")

print(f"\n✅ FASE 3 COMPLETADA: Datos listos para Machine Learning")
print(f"="*80)


📋 RESUMEN EJECUTIVO - PREPARACIÓN DE DATOS

📥 DATOS DE ENTRADA:
   • Dataset original: (99193, 15)
   • Período: 2020-2022 (COVID-19 Chile)
   • Calidad inicial: 0 columnas con valores faltantes

🔧 PROCESOS APLICADOS:
   1. ✅ Limpieza de datos:
      • Outliers: 5 variables tratadas
   2. ✅ Feature Engineering:
      • Features temporales: 16 creadas
      • Features tendencias/lags: 208 creadas
      • Features ratios: 3 creadas
   3. ✅ Targets ML:
      • Regresión: 4 targets
      • Clasificación: 4 targets
   4. ✅ Transformaciones:
      • Escalado robusto aplicado
      • Normalización Min-Max para variables cíclicas
      • Codificación de variables categóricas

📊 RESULTADOS FINALES:
   • Dataset final: (99193, 279)
   • Features para ML: 53
   • Train: (64475, 62)
   • Validation: (14879, 62)
   • Test: (19839, 62)

🎯 TARGETS DISPONIBLES:
   📈 Regresión:
      • target_cases_next_7_days: Suma de casos confirmados en próximos 7 días
      • target_growth_rate_14_days: Tasa de cr

## 📝 3.10 Documentación Final

### Variables y Datasets Generados

**📊 Datasets Finales:**
- `train_ml`: Dataset de entrenamiento con features y targets
- `val_ml`: Dataset de validación para ajuste de hiperparámetros
- `test_ml`: Dataset de prueba para evaluación final
- `df_final_transformado`: Dataset completo con todas las transformaciones

**🎯 Variables Target:**
- **Regresión (4 targets):**
  - `target_cases_next_7_days`: Casos próximos 7 días
  - `target_growth_rate_14_days`: Tasa crecimiento 14 días
  - `target_deaths_avg_7_days`: Promedio muertes 7 días
  - `target_volatility_14_days`: Volatilidad futura

- **Clasificación (4 targets):**
  - `target_high_transmission_period`: Alta transmisión (binario)
  - `target_alert_level`: Nivel alerta (multiclase)
  - `target_trend_direction`: Dirección tendencia (multiclase)
  - `target_hospital_saturation_risk`: Riesgo saturación (binario)

**🔧 Features Creadas:**
- **Temporales:** Variables cíclicas, períodos pandemia, días desde inicio
- **Tendencias:** Lags (1,3,7,14 días), rolling statistics, diferencias
- **Ratios:** Tasas epidemiológicas, ratios de crecimiento, volatilidad
- **Transformadas:** Escalado robusto, normalización, transformaciones log

**📋 Información de Metadatos:**
- `targets_reg_info`: Diccionario con información detallada de targets regresión
- `targets_class_info`: Diccionario con información detallada de targets clasificación
- `features_ml`: Lista de features seleccionadas para ML
- `categorias_features`: Categorización de features por tipo
- `prep_summary`: Resumen completo del proceso de preparación

---

**📈 Progreso del Proyecto:**
- ✅ Fase 1: Business Understanding (Completada)
- ✅ Fase 2: Data Understanding (Completada)
- ✅ Fase 3: Data Preparation (Completada)
- 🔄 Fase 4: Modeling (Siguiente)
- ⏳ Fase 5: Evaluation (Pendiente)
- ⏳ Fase 6: Deployment (Pendiente)

**Continuar con:** Desarrollo de pipelines de Machine Learning en Kedro

### Variables Creadas por Categoría

**Variables Temporales (16 features):**
- Básicas: year, month, day, day_of_week, day_of_year, week_of_year, quarter
- Cíclicas: month_sin, month_cos, day_of_week_sin, day_of_week_cos
- Especiales: is_weekend, is_month_start, is_month_end, is_quarter_start
- Pandemia: days_since_pandemic_start, pandemic_period

**Variables de Tendencias:**
- Lags: valores en t-1, t-3, t-7, t-14
- Rolling: medias móviles, desviaciones estándar, máximos, mínimos
- Diferencias: cambios absolutos y porcentuales
- Aceleración: segunda derivada de tendencias

**Variables de Ratios:**
- Epidemiológicos: CFR diario, CFR total, tasa de positividad
- Crecimiento: comparaciones temporales semanales
- Volatilidad: coeficientes de variación
- Intensidad: índices compuestos de actividad epidémica

### Calidad Final del Dataset

**Completitud:**
- Train: {train_ml.shape if train_ml is not None else 'N/A'} observaciones
- Validation: {val_ml.shape if val_ml is not None else 'N/A'} observaciones  
- Test: {test_ml.shape if test_ml is not None else 'N/A'} observaciones

**Features Seleccionadas:**
- Total features disponibles: {len(features_ml)}
- Features de alta calidad para ML
- Balance entre interpretabilidad y poder predictivo

**Targets Balanceados:**
- 4 problemas de regresión con distribuciones apropiadas
- 4 problemas de clasificación con clases balanceadas
- Targets orientados a necesidades reales del negocio

### Próximas Fases del Proyecto

**Fase 4: Modeling**
- Implementar modelos baseline
- Desarrollar modelos avanzados (Random Forest, XGBoost, etc.)
- Optimización de hiperparámetros
- Validación cruzada temporal

**Fase 5: Evaluation**
- Evaluación exhaustiva de modelos
- Análisis de importancia de features
- Interpretabilidad de resultados
- Selección del mejor modelo por target

**Fase 6: Deployment**
- Pipeline de inferencia
- Monitoreo de modelo
- Documentación para producción
- Mantenimiento y actualización

---

**Estado del Proyecto: FASE 3 COMPLETADA**

Los datos están completamente preparados para la fase de modelado. Se han creado {len(targets_reg_info) + len(targets_class_info)} targets de Machine Learning viables con features de alta calidad derivadas de análisis epidemiológico.

**Archivos Generados:**
- `01_business_understanding.ipynb` ✅
- `02_data_understanding.ipynb` ✅  
- `03_data_preparation.ipynb` ✅
- Datasets ML listos para la siguiente fase

**Cumplimiento de Rúbrica:**
- ✅ 3 Notebooks CRISP-DM obligatorios
- ✅ Pipeline data_engineering funcionando
- ✅ Feature engineering avanzado (80+ features)
- ✅ 8 targets ML justificados
- ✅ Transformaciones diferenciadas
- ✅ División temporal apropiada

El proyecto está listo para proceder con la fase de modelado.